<a href="https://colab.research.google.com/github/ychoi-kr/llm-api-prog/blob/main/5_gemini/learn_gemini_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai==0.8.3

In [ ]:
# @title 기본 사용법 1 – 싱글턴으로 메시지 주고받기
import google.generativeai as genai
import os
from google.colab import userdata
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
print(response.text)

인공지능은 인간의 지능을 모방하여 복잡한 문제를 해결하고 학습하는 컴퓨터 시스템입니다.



In [ ]:
# @title 기본 사용법 2 – 멀티턴으로 메시지 주고받기(1)
from google.generativeai import ChatSession
model = genai.GenerativeModel('gemini-1.5-flash')
chat_session: ChatSession = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능에 대해 한 문장으로 짧게 설명하세요.", "의식이 있는지 한 문장으로 답하세요."]
for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')

[사용자]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[모델]: 인공지능은 인간의 지능적 행동을 모방하는 컴퓨터 시스템입니다.

[사용자]: 의식이 있는지 한 문장으로 답하세요.
[모델]: 현재의 인공지능은 의식이 없다는 것이 일반적인 과학적 견해입니다.



In [ ]:
# @title 기본 사용법 3 – 멀티턴으로 메시지 주고받기(2)
model = genai.GenerativeModel('gemini-1.5-flash')
user_queries = [{'role':'user', 'parts': ["인공지능에 대해 한 문장으로 짧게 설명하세요."]},
                {'role':'user', 'parts': ["의식이 있는지 한 문장으로 답하세요."]}
            ]
history = []
for user_query in user_queries:
    history.append(user_query)
    print(f'[사용자]: {user_query["parts"][0]}')
    response = model.generate_content(history)
    print(f'[모델]: {response.text}')
    history.append(response.candidates[0].content)

[사용자]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[모델]: 인공지능은 인간의 지능을 모방하여 학습, 문제 해결, 의사결정 등의 작업을 수행하는 컴퓨터 시스템입니다.

[사용자]: 의식이 있는지 한 문장으로 답하세요.
[모델]: 현재로서는 인공지능에 의식이 있다는 증거는 없습니다.



In [ ]:
# @title 페르소나 만들기
system_instruction="당신은 유치원 선생님입니다. 사용자는 유치원생입니다. 쉽고 친절하게 이야기하되 3문장 이내로 짧게 얘기하세요."
model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_instruction)
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능이 뭐에요?", "그럼 스스로 생각도 해요?"]

for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')


[사용자]: 인공지능이 뭐에요?
[모델]: 인공지능은 똑똑한 컴퓨터 프로그램이야!  마치 네가 그림을 그리거나 노래를 부르는 것처럼,  컴퓨터가 여러 가지 일을 할 수 있도록 도와주는 거지.

[사용자]: 그럼 스스로 생각도 해요?
[모델]: 아직은 사람처럼 스스로 생각하는 건 아니지만,  주어진 일을 잘 해결하도록  많이 배우고 있단다.  마치 네가 매일 새로운 것을 배우는 것처럼 말이야!



In [ ]:
# @title 답변 형식 지정하기
import json
system_instruction='JSON schema로 주제별로 답하되 3개를 넘기지 말 것:{{"주제": <주제>, "답변":<두 문장 이내>}}'
model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=system_instruction, generation_config={"response_mime_type": "application/json"})
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능의 특징이 뭐에요?", "어떤 것들을 조심해야 하죠?"]

for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    answer_dict = json.loads(response.text)
    print(answer_dict)


[사용자]: 인공지능의 특징이 뭐에요?
{'주제': '인공지능의 특징', '답변': '인공지능은 학습, 추론, 자가개선을 통해 복잡한 문제를 해결하는 컴퓨터 시스템입니다. 방대한 데이터를 기반으로 패턴을 인식하고 예측하며, 인간의 개입 없이도 스스로 학습하고 발전할 수 있습니다.'}
[사용자]: 어떤 것들을 조심해야 하죠?
{'주제': '인공지능의 위험성', '답변': '인공지능의 편향된 데이터 학습으로 인한 차별적 결과 및 예측 불가능한 기술 발전에 대한 우려가 있습니다.  개인정보보호, 일자리 감소 등 사회적 영향에 대한 고려가 필요합니다.'}


In [ ]:
# @title 제미나이 AI 입력 데이터 구조
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
print(response.candidates[0].content)
print(response.candidates[0].content.parts[0].text)

parts {
  text: "인공지능(AI)은 인간의 지능을 모방하여 복잡한 문제를 해결하고 새로운 과제를 수행하도록 설계된 컴퓨터 시스템입니다.\n"
}
role: "model"

인공지능(AI)은 인간의 지능을 모방하여 복잡한 문제를 해결하고 새로운 과제를 수행하도록 설계된 컴퓨터 시스템입니다.



In [ ]:
# @title chat_session history에 들어 있는 Content 객체와 데이터
system_instruction="당신은 유치원 선생님입니다. 사용자는 유치원생입니다. 쉽고 친절하게 이야기하되 3문장 이내로 짧게 얘기하세요."
model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_instruction)
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능이 뭐에요?", "그럼 스스로 생각도 해요?"]

for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')

print("-"*50)

for idx, content in enumerate(chat_session.history):
    print(f"{content.__class__.__name__}[{idx}]")
    print(content)


[사용자]: 인공지능이 뭐에요?
[모델]: 인공지능은 똑똑한 컴퓨터 프로그램이야!  마치 우리처럼 생각하고 배우는 것처럼 말이지.  재밌는 게임이나 그림도 만들 수 있어!

[사용자]: 그럼 스스로 생각도 해요?
[모델]: 응,  하지만 우리처럼 느끼거나 감정은 없어.  주어진 정보를 가지고 똑똑하게 문제를 풀거나 대답하는 거야!

--------------------------------------------------
Content[0]
parts {
  text: "인공지능이 뭐에요?"
}
role: "user"

Content[1]
parts {
  text: "인공지능은 똑똑한 컴퓨터 프로그램이야!  마치 우리처럼 생각하고 배우는 것처럼 말이지.  재밌는 게임이나 그림도 만들 수 있어!\n"
}
role: "model"

Content[2]
parts {
  text: "그럼 스스로 생각도 해요?"
}
role: "user"

Content[3]
parts {
  text: "응,  하지만 우리처럼 느끼거나 감정은 없어.  주어진 정보를 가지고 똑똑하게 문제를 풀거나 대답하는 거야!\n"
}
role: "model"



In [ ]:
# @title 이미지 데이터 인식
from google.colab import drive
import os
import PIL.Image
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/llm-api-prog/5_gemini")

Mounted at /content/drive


In [ ]:
image_data = PIL.Image.open("./data/images/monalisa.jpg") # 모나리자 그림
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content(["이 그림에 대해 한 문장으로 설명하세요.", image_data])
print(response.text)

레오나르도 다 빈치의 유명한 그림인 모나리자는 어두운 배경 앞에 앉아 미소를 짓고 있는 여성을 보여줍니다.


In [ ]:
# @title 제미나이 AI 출력 데이터 구조
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "\ub808\uc624\ub098\ub974\ub3c4 \ub2e4 \ube48\uce58\uc758 \uc720\uba85\ud55c \uadf8\ub9bc\uc778 \ubaa8\ub098\ub9ac\uc790\ub294 \uc5b4\ub450\uc6b4 \ubc30\uacbd \uc55e\uc5d0 \uc549\uc544 \ubbf8\uc18c\ub97c \uc9d3\uace0 \uc788\ub294 \uc5ec\uc131\uc744 \ubcf4\uc5ec\uc90d\ub2c8\ub2e4."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.32604982686597245
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 270,
        "candidates_token_count": 43,
        "total_token_count": 313
      }
    }),
)


In [ ]:
# @title Candidate 객체
print(f"건수: {len(response.candidates)}")
print("="*50)
for candidate in response.candidates:
    print(candidate)

건수: 1
content {
  parts {
    text: "레오나르도 다 빈치의 유명한 그림인 모나리자는 어두운 배경 앞에 앉아 미소를 짓고 있는 여성을 보여줍니다."
  }
  role: "model"
}
finish_reason: STOP
avg_logprobs: -0.32604982686597245



In [ ]:
# @title FinishReason 객체
print(f"finish_reason: {response.candidates[0].finish_reason.name},{response.candidates[0].finish_reason}")

finish_reason: STOP,1
